In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import random
import pickle 
import matplotlib.pyplot as plt
from collections import Counter
import vist
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)

# Caltech Story Classification

In [2]:
with open("data/caltech_stories_dataset", 'rb') as file:
        caltech_stories_dataset = pickle.load(file)
        

type(caltech_stories_dataset)
labels = [1] * 12000
labels[7200:] = [0] * 4800

caltech_stories_dataset = np.array(caltech_stories_dataset)
labels = np.array(labels)

#shuffle and create datasets
indices = list(range(12000))
random.shuffle(indices)

labels = labels[indices]
caltech_stories_dataset = caltech_stories_dataset[indices]


train = {"features" : {"filenames" : caltech_stories_dataset[:10000,:]}, "labels" : labels[:10000]}
valid = {"features" : {"filenames" : caltech_stories_dataset[10000:11000,:]}, "labels" : labels[10000:11000]}
test = {"features" : {"filenames" : caltech_stories_dataset[11000:,:]}, "labels" : labels[11000:]}

caltech_data = {"training": train, "validation": valid, "test" : test}
print("Labeled and segmented the dataset into training/validation/test sets")

Labeled and segmented the dataset into training/validation/test sets


In [3]:
with open("data/caltech_features", 'rb') as file:
        caltech_features = pickle.load(file)
        
        
for _set in caltech_data:
    features = caltech_data[_set]["features"]
    filenames = features["filenames"]
    features["fc7_features"] = []
    features["fc8_features"] = []
    
    for story in filenames:
        
        fc7_features_story = []
        fc8_features_story = []
        
        for img_filename in story:
            tokens = img_filename.split("/")
            class_name = tokens[-2]
            img_name = tokens[-1] 
            
            fc7 = caltech_features[class_name][img_name]["fc7"]
            fc8 = caltech_features[class_name][img_name]["fc8"]
            
            fc7_features_story.append(fc7) 
            fc8_features_story.append(fc8)
        
        features["fc7_features"].append(fc7_features_story) 
        features["fc8_features"].append(fc8_features_story)

print("Loaded fc7 and fc8 features")

Loaded fc7 and fc8 features


In [4]:
caltech_data.keys()

dict_keys(['test', 'validation', 'training'])

In [5]:
caltech_data['training'].keys()

dict_keys(['features', 'labels'])

In [6]:
caltech_data['training']['features'].keys()

dict_keys(['fc8_features', 'fc7_features', 'filenames'])

In [7]:
len(caltech_data['test']['features']['fc8_features'][0])

5

# LSTM

In [8]:
def get_batch(iterable, n=1):
    """
    Yields an iteratable
    """
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        

def create_batches(data, batch_size = 100):
    
    batches = list()
    for batch in get_batch(data, batch_size):
        batches.append(batch)
    
    return batches

In [9]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 100
# display_step = 10

# Network Parameters
n_input = 1000 # MNIST data input (img shape: vector 1000)
n_steps = 5 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 2  # true false

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [10]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [14]:
# Launch the graph
with tf.Session() as sess:
    
    # create batches for data and labels 
    data = caltech_data['training']['features']['fc8_features']
    data_labels = caltech_data['training']['labels']
    batches_x = create_batches(data, batch_size)
    batches_y = create_batches(data_labels, batch_size)

    num_of_batches = np.shape(batches_x)[0]
    
    sess.run(init)
    step = 1
    epochs = 2

    # Keep training until reach max iterations
    while step  < epochs:
        
        for i in range(num_of_batches):
            # transform x
            batch_x = np.zeros((100, 5000))
            r = 0
            for story_imgs in batches_x[0]:
                # initialize empty vector
                row = np.empty(0)
                for img_features in story_imgs:
                    row = np.append(row, img_features)

                batch_x[r, :] = row
                r = r + 1
            #
            batch_x = batch_x.reshape((100, 5, 1000))
            
            # labels to one hot representation
            batch_y = np.zeros((100, 2))
            r = 0
            for label in batches_y[0]:
                if label == 0:
                    batch_y[r, :] = np.array([1,0])
                else:
                    batch_y[r, :] = np.array([0,1])

                r = r + 1
            #
            
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Batch " + str(i) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
            step += 1
        print ("Optimization Finished!")

#     # Calculate accuracy for 128 mnist test images
#     test_len = 128
#     test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
#     test_label = mnist.test.labels[:test_len]
#     print ("Testing Accuracy:", \
#         sess.run(accuracy, feed_dict={x: test_data, y: test_label})

Batch 0, Minibatch Loss= 6.778980, Training Accuracy= 0.56000
Batch 1, Minibatch Loss= 3.243359, Training Accuracy= 0.77000
Batch 2, Minibatch Loss= 1.768178, Training Accuracy= 0.89000
Batch 3, Minibatch Loss= 1.024702, Training Accuracy= 0.93000
Batch 4, Minibatch Loss= 0.580544, Training Accuracy= 0.96000
Batch 5, Minibatch Loss= 0.248796, Training Accuracy= 0.96000
Batch 6, Minibatch Loss= 0.033994, Training Accuracy= 0.99000
Batch 7, Minibatch Loss= 0.001466, Training Accuracy= 1.00000
Batch 8, Minibatch Loss= 0.000028, Training Accuracy= 1.00000
Batch 9, Minibatch Loss= 0.000004, Training Accuracy= 1.00000
Batch 10, Minibatch Loss= 0.000046, Training Accuracy= 1.00000
Batch 11, Minibatch Loss= 0.000370, Training Accuracy= 1.00000
Batch 12, Minibatch Loss= 0.001194, Training Accuracy= 1.00000
Batch 13, Minibatch Loss= 0.001913, Training Accuracy= 1.00000
Batch 14, Minibatch Loss= 0.001718, Training Accuracy= 1.00000
Batch 15, Minibatch Loss= 0.001335, Training Accuracy= 1.00000
Ba

### MNIST example

In [5]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.938751, Training Accuracy= 0.32031
Iter 2560, Minibatch Loss= 1.539190, Training Accuracy= 0.47656
Iter 3840, Minibatch Loss= 1.279017, Training Accuracy= 0.61719
Iter 5120, Minibatch Loss= 1.198248, Training Accuracy= 0.58594
Iter 6400, Minibatch Loss= 0.941516, Training Accuracy= 0.71094
Iter 7680, Minibatch Loss= 0.715814, Training Accuracy= 0.75781
Iter 8960, Minibatch Loss= 0.527712, Training Accuracy= 0.85938
Iter 10240, Minibatch Loss= 0.462727, Training Accuracy= 0.85156
Iter 11520, Minibatch Loss= 0.492773, Training Accuracy= 0.84375
Iter 12800, Minibatch Loss= 0.425529, Training Accuracy= 0.84375
Iter 14080, Minibatch Loss= 0.402597, Training Accuracy= 0.88281
Iter 15360, Minibatch Loss= 0.635651, Training Accuracy= 0.78125
Iter 16640, Minibatch Loss= 0.342599, Training Accuracy= 0.89062
Iter 17920, Minibatch Loss= 0.377497, Training Accuracy= 0.84375
Iter 19200, Minibatch Loss= 0.350818, Training Accuracy= 0.85938
Iter 20480, Minibatch Loss= 0.29